# Test notebook to exercise the Lasair API

Important: change this to the correct import!

In [20]:
import sys
sys.path.append('../../lasair/lasair_api')

Change this to the API token and endpoint you want to test

In [21]:
lsst=False # if you change this restart kernel
if lsst:
    sys.path.append('API_lsst')
    endpoint = "https://lasair-lsst.lsst.ac.uk/api"
    oid = 'diaObjectId'
    raname = 'ra'
    decname = 'decl'
else:
    sys.path.append('API_ztf')
    endpoint = "https://lasair-ztf.lsst.ac.uk/api"
    oid = 'objectId'
    raname = 'ramean'
    decname = 'decmean'

Fetch your API TOKEN from your Lasair account, top right under "profile"

In [22]:
import json
from lasair import LasairError, lasair_client as lasair
import settings
L = lasair(settings.API_TOKEN, endpoint=endpoint)

## Cone search

In [23]:
racone  = 279.22
deccone = -47.91
radius = 60 # arcsec
try:
    result = L.cone(racone, deccone, radius, requestType='nearest')
    print(json.dumps(result, indent=2))
except:
    print('nothing found')

nothing found


## Database query

In [24]:
selected = 'objects.%s, objects.%s, objects.%s' % (oid, raname, decname)
tables = 'objects,sherlock_classifications'
conditions = 'classification="SN"'
results = L.query(selected, tables, conditions, limit = 10)

for row in results:
    objectId = row[oid]
    ra = row[raname]
    dec = row[decname]
    print(objectId, ra, dec)

# Just use this objectId in subsequent calls
objectId = str(objectId)

ZTF17aaaadix 75.1234974 44.3545903
ZTF17aaaadqv 71.608009725 44.112756225000005
ZTF17aaaafup 82.7761947 23.7206712
ZTF17aaaajfc 325.68412294999996 47.153728099999995
ZTF17aaaazvg 324.92064404 45.6440102
ZTF17aaabcyg 325.2470203 48.2919921
ZTF17aaabjzl 83.2786894 24.4758433
ZTF17aaablao 58.854258099999996 45.597521900000004
ZTF17aaabneq 86.1023892375 33.6711469625
ZTF17aaabolo 93.97128599999999 26.589407299999998


## Lightcurves

In [25]:
result = L.lightcurves([objectId])
print(json.dumps(result, indent=2))

[
  {
    "objectId": "ZTF17aaabolo",
    "candidates": [
      {
        "candid": 683360713415015053,
        "jd": 2458437.8607176,
        "ra": 93.9713126,
        "dec": 26.5893983,
        "fid": 2,
        "nid": 683,
        "magpsf": 19.496400833129883,
        "sigmapsf": 0.14922399818897247,
        "magnr": 21.604000091552734,
        "sigmagnr": 0.13300000131130219,
        "magzpsci": 26.334400177001953,
        "isdiffpos": "t",
        "ssdistnr": -999.0,
        "ssnamenr": "null",
        "drb": null
      },
      {
        "candid": 683433853415015048,
        "jd": 2458437.9338542,
        "ra": 93.9712676,
        "dec": 26.5893294,
        "fid": 1,
        "nid": 683,
        "magpsf": 19.849899291992188,
        "sigmapsf": 0.1916539967060089,
        "magnr": 21.559999465942383,
        "sigmagnr": 0.09200000017881393,
        "magzpsci": 26.313899993896484,
        "isdiffpos": "t",
        "ssdistnr": -999.0,
        "ssnamenr": "null",
        "drb": null


## Sherlock Object

In [26]:
lite=True
result = L.sherlock_object(objectId, lite=lite)
print(json.dumps(result, indent=2))

{
  "classifications": {
    "ZTF17aaabolo": [
      "SN",
      "The transient is possibly associated with <em>139900939712637877</em>; an r=22.54 mag galaxy found in the PS1 catalogue. It's located 0.03\" N, 0.02\" E from the galaxy centre."
    ]
  },
  "crossmatches": [
    {
      "transient_object_id": "ZTF17aaabolo",
      "association_type": "SN",
      "catalogue_table_name": "PS1",
      "catalogue_object_id": "139900939712637877",
      "catalogue_object_type": "galaxy",
      "raDeg": 93.97127983,
      "decDeg": 26.589398834,
      "separationArcsec": 0.036,
      "northSeparationArcsec": "-0.0305",
      "eastSeparationArcsec": "-0.0199",
      "physical_separation_kpc": null,
      "direct_distance": null,
      "distance": null,
      "z": null,
      "photoZ": null,
      "photoZErr": null,
      "Mag": 22.541,
      "MagFilter": "r",
      "MagErr": 0.011,
      "classificationReliability": 2,
      "major_axis_arcsec": null
    }
  ]
}


## Sherlock Position

In [27]:
result = L.sherlock_position(ra, dec, lite=lite)
print(json.dumps(result, indent=2))

{
  "classifications": {
    "transient_00000": [
      "SN",
      "The transient is possibly associated with <em>139900939712637877</em>; an r=22.54 mag galaxy found in the PS1 catalogue. It's located 0.03\" N, 0.02\" E from the galaxy centre."
    ]
  },
  "crossmatches": [
    {
      "transient_object_id": "transient_00000",
      "association_type": "SN",
      "catalogue_table_name": "PS1",
      "catalogue_object_id": "139900939712637877",
      "catalogue_object_type": "galaxy",
      "raDeg": 93.97127983,
      "decDeg": 26.589398834,
      "separationArcsec": 0.036,
      "northSeparationArcsec": "-0.03048",
      "eastSeparationArcsec": "-0.01986",
      "physical_separation_kpc": null,
      "direct_distance": null,
      "distance": null,
      "z": null,
      "photoZ": null,
      "photoZErr": null,
      "Mag": 22.541,
      "MagFilter": "r",
      "MagErr": 0.011,
      "classificationReliability": 2,
      "major_axis_arcsec": null
    }
  ]
}


## All object data

In [28]:
result = L.objects(objectId)
print(json.dumps(result, indent=2))

[
  {
    "objectId": "ZTF17aaabolo",
    "objectData": {
      "ncand": 2,
      "ramean": 93.97128599999999,
      "decmean": 26.589407299999998,
      "glonmean": 185.3587949712237,
      "glatmean": 4.65350184433523,
      "jdmin": 2460364.6948611,
      "jdmax": 2460368.7593634,
      "rasex": "06:15:53.109",
      "decsex": "26:35:21.866",
      "ec_lon": 93.55624949471026,
      "ec_lat": 3.199127218026181,
      "now_mjd": "60545.53",
      "mjdmin_ago": 181.3362427491229,
      "mjdmax_ago": 177.27174044935964,
      "discMjd": 58437.36071760021,
      "discUtc": "2018-11-15 08:39:26",
      "discMag": "19.50\u00b10.15",
      "discFilter": "r",
      "latestMjd": 60368.25936339982,
      "latestUtc": "2024-02-28 06:13:28",
      "latestMag": "18.94\u00b10.12",
      "latestFilter": "r",
      "peakMjd": 60364.194861100055,
      "peakUtc": "2024-02-24 04:40:35",
      "peakMag": "18.16\u00b10.24",
      "peakFilter": "r"
    },
    "candidates": [
      {
        "candid": 26